In [1]:
from typing import Union

from vivarium.core.types import State, Update

from pbg.translate import get_port_mapping
from pbg.parse import get_process, find_defaults
from process_bigraph import ProcessTypes, pp
from bigraph_schema.type_system import required_schema_keys, optional_schema_keys
from ecoli.processes.antibiotics.cell_wall import CellWall
from vivarium.core.registry import process_registry
from vivarium.core.process import Process as VivariumProcess
from ecoli.processes.registries import topology_registry
from vivarium_interface.vivarium import Vivarium
from pbg.data_model.base_process import BaseProcess, CORE
from pbg.parse import find_defaults

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


In [2]:
class Add(BaseProcess):
    defaults = {
        'k': 0.11 / 2.22
    }
    name = 'add'

    def __init__(self, parameters=None, core=None):
        super().__init__(parameters, core=core)

    def ports_schema(self):
        return {
            'x': {
                '_default': -0.11,
                '_emit': True,
            },
            'y': {
                '_default': 0.22,
                '_emit': True,
            }
        }

    def next_update(
            self, timestep, states):
        x0 = states['x']
        x = x0 * self.parameters['k'] * timestep
        return {
            'x': x,
            'y': x0 + (x * timestep)
        }

In [3]:
core = ProcessTypes()
types: dict = core.types()
processes = core.process_registry.registry
processes.update(process_registry.registry)
processes.update({'Add': Add})
type_keys: set = required_schema_keys.union(optional_schema_keys)

In [4]:
viv: Vivarium = Vivarium(types=types, processes=processes)

In [5]:
viv.add_process(process_id='Add', name='add', config={'k': -11.11})

In [6]:
proc: BaseProcess = viv.get_state()['add']['instance']
state = find_defaults(proc.ports_schema())

state

{'x': -0.11, 'y': 0.22}

In [8]:
pp(viv)

Vivarium( 
{ 'add': { 'address': 'local:Add',
           'config': {'k': -11.11, 'timestep': 1.0},
           'inputs': {},
           'instance': <__main__.Add object at 0x1324bd820>,
           'interval': 1.0,
           'outputs': {}},
  'global_time': 0.0})


In [7]:
viv.connect_process(
    process_name='add',
    inputs={
        'x': ['x']
    },
    outputs={
        'y': ['y']
    }
)

In [9]:
for k, v in state.items():
    viv.set_value(path=k, value=v)

In [10]:
pp(viv)

Vivarium( 
{ 'add': { 'address': 'local:Add',
           'config': {'k': -11.11, 'timestep': 1.0},
           'inputs': {'x': ['x']},
           'instance': <__main__.Add object at 0x15f9eb440>,
           'interval': 1.0,
           'outputs': {'y': ['y']}},
  'global_time': 0.0,
  'x': -0.11,
  'y': 0.22})


In [11]:
viv.add_emitter()

In [12]:
viv.run(10)
viv.get_results()

[{'global_time': 0.0, 'x': -0.11, 'y': 0.22},
 {'global_time': 1.0, 'x': -0.11, 'y': 1.3320999999999998},
 {'global_time': 2.0, 'x': -0.11, 'y': 2.4441999999999995},
 {'global_time': 3.0, 'x': -0.11, 'y': 3.5562999999999994},
 {'global_time': 4.0, 'x': -0.11, 'y': 4.668399999999999},
 {'global_time': 5.0, 'x': -0.11, 'y': 5.780499999999999},
 {'global_time': 6.0, 'x': -0.11, 'y': 6.892599999999999},
 {'global_time': 7.0, 'x': -0.11, 'y': 8.0047},
 {'global_time': 8.0, 'x': -0.11, 'y': 9.1168},
 {'global_time': 9.0, 'x': -0.11, 'y': 10.2289},
 {'global_time': 10.0, 'x': -0.11, 'y': 11.341}]

In [13]:
proc.initial_state()

{'x': -0.11, 'y': 0.22}

In [14]:
proc.update(state=proc.initial_state(), interval=2)

{'x': 2.4442, 'y': 4.7783999999999995}

In [15]:
viv.run(10)
viv.get_results()

[{'global_time': 0.0, 'x': -0.11, 'y': 0.22},
 {'global_time': 1.0, 'x': -0.11, 'y': 1.3320999999999998},
 {'global_time': 2.0, 'x': -0.11, 'y': 2.4441999999999995},
 {'global_time': 3.0, 'x': -0.11, 'y': 3.5562999999999994},
 {'global_time': 4.0, 'x': -0.11, 'y': 4.668399999999999},
 {'global_time': 5.0, 'x': -0.11, 'y': 5.780499999999999},
 {'global_time': 6.0, 'x': -0.11, 'y': 6.892599999999999},
 {'global_time': 7.0, 'x': -0.11, 'y': 8.0047},
 {'global_time': 8.0, 'x': -0.11, 'y': 9.1168},
 {'global_time': 9.0, 'x': -0.11, 'y': 10.2289},
 {'global_time': 10.0, 'x': -0.11, 'y': 11.341},
 {'global_time': 11.0, 'x': -0.11, 'y': 12.4531},
 {'global_time': 12.0, 'x': -0.11, 'y': 13.565199999999999},
 {'global_time': 13.0, 'x': -0.11, 'y': 14.677299999999999},
 {'global_time': 14.0, 'x': -0.11, 'y': 15.789399999999999},
 {'global_time': 15.0, 'x': -0.11, 'y': 16.9015},
 {'global_time': 16.0, 'x': -0.11, 'y': 18.013599999999997},
 {'global_time': 17.0, 'x': -0.11, 'y': 19.125699999999995}

In [16]:
proc.name

'add'